<a href="https://colab.research.google.com/github/FCW-UCL/CASA0025-WORKSHOP/blob/main/Copy_of_W05_quiz_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 Quiz

This notebook contains the SQL Quiz for Week 5. Section 1 uses the New York City data we know and love. Section 2 uses building footprint data from the Google Open Buildings dataset.

INSTRUCTIONS:

Run this notebook in Google Colab. The answer to each quesiton will be a number or a string. Input these into the corresponding question on Moodle. You have 90 minutes to attempt the quiz, so if you get stuck on a question, move on.

Make sure you run all of the cells of code in order, especially the ones that already have code in them! If you run into serious problems, try clicking on the "runtime" tab above and selecting "restart session and run all".
Use the cells below for any necessary setup.

In [1]:
!apt-get update
!apt-get install -y postgresql postgresql-contrib postgis
!pip install psycopg2-binary sqlalchemy geoalchemy2

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,725 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,683 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,894 kB]
Ge

In [2]:
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
!sudo -u postgres psql -c "CREATE DATABASE gis;"
!sudo -u postgres psql -d gis -c "CREATE EXTENSION postgis;"

CREATE DATABASE
CREATE EXTENSION


In [24]:
%load_ext sql
%sql postgresql://postgres@localhost/gis

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sqlalchemy/engine/base.py", line 143, in __init__
    self._dbapi_connection = engine.raw_connection()
                             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sqlalchemy/engine/base.py", line 3309, in raw_connection
    return self.pool.connect()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sqlalchemy/pool/base.py", line 447, in connect
    return _ConnectionFairy._checkout(self)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sqlalchemy/pool/base.py", line 1264, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sqlalchemy/pool/base.py", line 711, in checkout
    rec = pool._do_get()
          ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sqlalchemy/pool/impl.py", lin

# Section 1



The URL below points to a zip file containing the data on New York City that we've been working with so far.

https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip

## Question 1

Create the following tables using the corresponding shapefiles.
- nyc_neighborhoods
- nyc_census_blocks
- nyc_homicides
- nyc_streets
- nyc_subway_stations

In [5]:
!wget -O postgis-workshop-2020.zip https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip
!unzip -o postgis-workshop-2020.zip -d nyc_data
!find nyc_data -maxdepth 3 -type f -name "*.shp" | head -n 20

--2026-02-10 15:33:35--  https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.165.136, 52.216.208.72, 52.217.234.80, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.165.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22547324 (22M) [application/zip]
Saving to: ‘postgis-workshop-2020.zip’

postgis-workshop-20 100%[===================>]  21.50M  27.7MB/s    in 0.8s    

2026-02-10 15:33:36 (27.7 MB/s) - ‘postgis-workshop-2020.zip’ saved [22547324/22547324]

Archive:  postgis-workshop-2020.zip
   creating: nyc_data/postgis-workshop/
   creating: nyc_data/postgis-workshop/printing/
  inflating: nyc_data/postgis-workshop/printing/nyc_data_dictionary.pdf  
  inflating: nyc_data/postgis-workshop/printing/postgis-workshop-exercises.docx  
  inflating: nyc_data/postgis-workshop/printing/postgis-workshop-exercises.pdf  
  inflating: nyc_data/postgis-workshop/printing/nyc_data_dic

In [6]:
!find nyc_data -type f -name "*neigh*.shp"
!find nyc_data -type f -name "*census*.shp"
!find nyc_data -type f -name "*homic*.shp"
!find nyc_data -type f -name "*street*.shp"
!find nyc_data -type f -name "*subway*.shp"

nyc_data/postgis-workshop/data/nyc_neighborhoods.shp
nyc_data/postgis-workshop/data/nyc_census_blocks.shp
nyc_data/postgis-workshop/data/2000/nyc_census_blocks_2000.shp
nyc_data/postgis-workshop/data/nyc_homicides.shp
nyc_data/postgis-workshop/data/nyc_streets.shp
nyc_data/postgis-workshop/data/nyc_subway_stations.shp


In [8]:
!sudo -u postgres psql -d gis -c "DROP TABLE IF EXISTS nyc_neighborhoods, nyc_census_blocks, nyc_homicides, nyc_streets, nyc_subway_stations CASCADE;"

SRID=26918

!shp2pgsql -I -s $SRID "nyc_data/postgis-workshop/data/nyc_neighborhoods.shp" public.nyc_neighborhoods | sudo -u postgres psql -d gis
!shp2pgsql -I -s $SRID "nyc_data/postgis-workshop/data/nyc_census_blocks.shp" public.nyc_census_blocks | sudo -u postgres psql -d gis
!shp2pgsql -I -s $SRID "nyc_data/postgis-workshop/data/nyc_homicides.shp" public.nyc_homicides | sudo -u postgres psql -d gis
!shp2pgsql -I -s $SRID "nyc_data/postgis-workshop/data/nyc_streets.shp" public.nyc_streets | sudo -u postgres psql -d gis
!shp2pgsql -I -s $SRID "nyc_data/postgis-workshop/data/nyc_subway_stations.shp" public.nyc_subway_stations | sudo -u postgres psql -d gis


Streaming output truncated to the last 5000 lines.
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INS

## Question 2:
How many subway stations are there in NYC?



In [12]:
!sudo -u postgres psql -d gis -P pager=off -c "SELECT COUNT(*) FROM nyc_subway_stations;"

 count 
-------
   491
(1 row)



## Question 3:

What combination of borough and year hard the largest number of homicide victims?

In [14]:
!sudo -u postgres psql -d gis -P pager=off -c "\d nyc_homicides"

                                      Table "public.nyc_homicides"
   Column   |         Type          | Collation | Nullable |                  Default                   
------------+-----------------------+-----------+----------+--------------------------------------------
 gid        | integer               |           | not null | nextval('nyc_homicides_gid_seq'::regclass)
 incident_d | date                  |           |          | 
 boroname   | character varying(13) |           |          | 
 num_victim | character varying(1)  |           |          | 
 primary_mo | character varying(20) |           |          | 
 id         | double precision      |           |          | 
 weapon     | character varying(16) |           |          | 
 light_dark | character varying(1)  |           |          | 
 year       | double precision      |           |          | 
 geom       | geometry(Point,26918) |           |          | 
Indexes:
    "nyc_homicides_pkey" PRIMARY KEY, btree (gid)
  

In [15]:
!sudo -u postgres psql -d gis -P pager=off -c "\d nyc_neighborhoods"

                                        Table "public.nyc_neighborhoods"
  Column  |             Type             | Collation | Nullable |                    Default                     
----------+------------------------------+-----------+----------+------------------------------------------------
 gid      | integer                      |           | not null | nextval('nyc_neighborhoods_gid_seq'::regclass)
 boroname | character varying(43)        |           |          | 
 name     | character varying(64)        |           |          | 
 geom     | geometry(MultiPolygon,26918) |           |          | 
Indexes:
    "nyc_neighborhoods_pkey" PRIMARY KEY, btree (gid)
    "nyc_neighborhoods_geom_idx" gist (geom)



In [17]:
!sudo -u postgres psql -d gis -P pager=off -c "SELECT boroname, year, COUNT(*) AS victims FROM nyc_homicides GROUP BY boroname, year ORDER BY victims DESC LIMIT 1;"

 boroname | year | victims 
----------+------+---------
 Brooklyn | 2003 |     218
(1 row)



## Question 4:

Which subway station has the largest number of shooting victims within a 300 meter radius?


In [19]:
!sudo -u postgres psql -d gis -P pager=off -c "SELECT weapon, COUNT(*) FROM nyc_homicides GROUP BY weapon ORDER BY COUNT(*) DESC;"


      weapon      | count 
------------------+-------
 gun              |  2443
 knife            |   810
                  |   463
 blunt_instrument |   168
 other            |   100
(5 rows)



In [20]:
!sudo -u postgres psql -d gis -P pager=off -c "\d nyc_subway_stations"

                                     Table "public.nyc_subway_stations"
  Column   |         Type          | Collation | Nullable |                     Default                      
-----------+-----------------------+-----------+----------+--------------------------------------------------
 gid       | integer               |           | not null | nextval('nyc_subway_stations_gid_seq'::regclass)
 objectid  | numeric               |           |          | 
 id        | numeric               |           |          | 
 name      | character varying(31) |           |          | 
 alt_name  | character varying(38) |           |          | 
 cross_st  | character varying(27) |           |          | 
 long_name | character varying(60) |           |          | 
 label     | character varying(50) |           |          | 
 borough   | character varying(15) |           |          | 
 nghbhd    | character varying(30) |           |          | 
 routes    | character varying(20) |           |  

In [22]:
!sudo -u postgres psql -d gis -P pager=off -c "SELECT s.name, SUM(COALESCE(NULLIF(h.num_victim,'')::int,1)) AS shooting_victims FROM nyc_subway_stations s JOIN nyc_homicides h ON ST_DWithin(s.geom,h.geom,300) WHERE h.weapon='gun' GROUP BY s.name ORDER BY shooting_victims DESC LIMIT 1;"


     name     | shooting_victims 
--------------+------------------
 Franklin Ave |               28
(1 row)



## Question 5

what is the most densely populated residential street in new york? (total population of intersecting census blocks divided by the total length of the street)

In [25]:
!sudo -u postgres psql -d gis -P pager=off -c "\d nyc_streets"

                                         Table "public.nyc_streets"
 Column |              Type               | Collation | Nullable |                 Default                  
--------+---------------------------------+-----------+----------+------------------------------------------
 gid    | integer                         |           | not null | nextval('nyc_streets_gid_seq'::regclass)
 id     | double precision                |           |          | 
 name   | character varying(200)          |           |          | 
 oneway | character varying(10)           |           |          | 
 type   | character varying(50)           |           |          | 
 geom   | geometry(MultiLineString,26918) |           |          | 
Indexes:
    "nyc_streets_pkey" PRIMARY KEY, btree (gid)
    "nyc_streets_geom_idx" gist (geom)



In [26]:
!sudo -u postgres psql -d gis -P pager=off -c "SELECT type, COUNT(*) FROM nyc_streets GROUP BY type ORDER BY COUNT(*) DESC;"

                       type                       | count 
--------------------------------------------------+-------
 residential                                      | 16560
 motorway_link                                    |   953
 footway                                          |   345
 unclassified                                     |   337
 tertiary                                         |   257
 motorway                                         |   227
 secondary                                        |   137
 primary                                          |    98
 service                                          |    90
 pedestrian                                       |    20
 trunk_link                                       |    15
 trunk                                            |    14
 primary_link                                     |    12
 construction                                     |     8
 steps                                            |     7
 cycleway   

In [28]:
!sudo -u postgres psql -d gis -P pager=off -c "WITH streets AS (SELECT name, ST_UnaryUnion(ST_Collect(geom)) AS geom, SUM(ST_Length(geom)) AS len_m FROM nyc_streets WHERE type LIKE '%residential%' AND name IS NOT NULL AND name <> '' GROUP BY name), street_block AS (SELECT DISTINCT s.name, b.gid AS block_gid, s.len_m, b.popn_total AS pop FROM streets s JOIN nyc_census_blocks b ON ST_Intersects(s.geom, b.geom)) SELECT name, SUM(pop)/NULLIF(MAX(len_m),0) AS pop_per_meter FROM street_block GROUP BY name ORDER BY pop_per_meter DESC LIMIT 1;"

  name   |   pop_per_meter    
---------+--------------------
 W 66 St | 3223.7874796379137
(1 row)



# Section 2

This section uses internet speed data from [Ookla](https://github.com/teamookla/ookla-open-data); take a minute to read the documentation of the dataset.

Create a table called ookla_nyc which reads the data from this url:

https://storage.googleapis.com/qm2/CASA0025/ookla_nyc.parquet




In [29]:
!pip -q install duckdb pandas pyarrow sqlalchemy psycopg2-binary

In [30]:
import duckdb
import pandas as pd

url = "https://storage.googleapis.com/qm2/CASA0025/ookla_nyc.parquet"

con = duckdb.connect()
df = con.execute(f"SELECT * FROM read_parquet('{url}')").df()

df.shape, df.columns

((7645, 14),
 Index(['quadkey', 'tile_x', 'tile_y', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms',
        'avg_lat_down_ms', 'avg_lat_up_ms', 'tests', 'devices', 'quarter',
        'type', 'year', 'geometry'],
       dtype='object'))

In [36]:
type(df.loc[0, "geometry"]), df.loc[0, "geometry"][:80]

(str,
 'POLYGON ((559335.820448 4533081.030772, 559798.18087 4533084.773678, 559801.9238')

In [37]:
df["geometry"].head(3)

,geometry
0,"POLYGON ((559335.820448 4533081.030772, 559798..."
1,"POLYGON ((559798.18087 4533084.773678, 560260...."
2,"POLYGON ((560260.541371 4533088.54564, 560722...."


In [38]:
csv_path = "/tmp/ookla_nyc_raw.csv"
df.to_csv(csv_path, index=False)
csv_path, df.shape

('/tmp/ookla_nyc_raw.csv', (7645, 14))

In [41]:
%%bash
sudo -u postgres psql -d gis -P pager=off <<'SQL'

DROP TABLE IF EXISTS ookla_nyc_raw;

CREATE TABLE ookla_nyc_raw (
  quadkey text,
  tile_x double precision,
  tile_y double precision,
  avg_d_kbps double precision,
  avg_u_kbps double precision,
  avg_lat_ms double precision,
  avg_lat_down_ms double precision,
  avg_lat_up_ms double precision,
  tests int,
  devices int,
  quarter text,
  type text,
  year int,
  geometry text
);

\copy ookla_nyc_raw FROM '/tmp/ookla_nyc_raw.csv' WITH (FORMAT csv, HEADER true);

SELECT COUNT(*) FROM ookla_nyc_raw;

SQL

DROP TABLE
CREATE TABLE
COPY 7645
 count 
-------
  7645
(1 row)



In [42]:
%%bash
sudo -u postgres psql -d gis -P pager=off <<'SQL'

DROP TABLE IF EXISTS ookla_nyc;

CREATE TABLE ookla_nyc AS
SELECT
  quadkey, tile_x, tile_y,
  avg_d_kbps, avg_u_kbps,
  avg_lat_ms, avg_lat_down_ms, avg_lat_up_ms,
  tests, devices, quarter, type, year,
  ST_SetSRID(ST_GeomFromText(geometry), 26918) AS geom
FROM ookla_nyc_raw;

CREATE INDEX ookla_nyc_geom_idx ON ookla_nyc USING GIST (geom);
ANALYZE ookla_nyc;

SELECT COUNT(*) FROM ookla_nyc;

SQL

DROP TABLE
SELECT 7645
CREATE INDEX
ANALYZE
 count 
-------
  7645
(1 row)



## Question 6
How many tiles are contained within new york city neighbourhoods?

In [43]:
%%bash
sudo -u postgres psql -d gis -P pager=off -c "
SELECT COUNT(*) AS n_tiles
FROM ookla_nyc o
JOIN nyc_neighborhoods n
  ON ST_Within(o.geom, n.geom);
"

 n_tiles 
---------
    1403
(1 row)



## Question 7

Which neighbourhood has the highest average download speed? use 'within' rather than intersects, and ignore tiles with fewer than 3 tests.

In [44]:
%%bash
sudo -u postgres psql -d gis -P pager=off -c "
SELECT
  n.name,
  AVG(o.avg_d_kbps) AS avg_download_kbps
FROM ookla_nyc o
JOIN nyc_neighborhoods n
  ON ST_Within(o.geom, n.geom)
WHERE o.tests >= 3
GROUP BY n.name
ORDER BY avg_download_kbps DESC
LIMIT 1;
"

   name   | avg_download_kbps 
----------+-------------------
 Woodside | 455960.3333333333
(1 row)



## Question 8



What is the name of the subway station closest to the tile with the highest download speed?

In [45]:
%%bash
sudo -u postgres psql -d gis -P pager=off -c "
WITH fastest AS (
  SELECT geom
  FROM ookla_nyc
  ORDER BY avg_d_kbps DESC
  LIMIT 1
)
SELECT s.name
FROM nyc_subway_stations s, fastest f
ORDER BY ST_Distance(s.geom, f.geom)
LIMIT 1;
"

   name   
----------
 190th St
(1 row)



## Question 9

Which neighbourhood has the highest number of devices per capita? Use only spatial intersections.

In [46]:
%%bash
sudo -u postgres psql -d gis -P pager=off -c "

WITH devices_per_nbhd AS (
  SELECT
    n.name,
    SUM(o.devices) AS total_devices
  FROM ookla_nyc o
  JOIN nyc_neighborhoods n
    ON ST_Intersects(o.geom, n.geom)
  GROUP BY n.name
),

population_per_nbhd AS (
  SELECT
    n.name,
    SUM(b.popn_total) AS total_population
  FROM nyc_census_blocks b
  JOIN nyc_neighborhoods n
    ON ST_Intersects(b.geom, n.geom)
  GROUP BY n.name
)

SELECT
  d.name,
  d.total_devices::float / p.total_population AS devices_per_capita
FROM devices_per_nbhd d
JOIN population_per_nbhd p
  ON d.name = p.name
WHERE p.total_population > 0
ORDER BY devices_per_capita DESC
LIMIT 1;

"

  name  | devices_per_capita 
--------+--------------------
 Corona | 0.4864864864864865
(1 row)



## Question 10


What is the average upload speed for tiles that are within 200 meters of L train stops and intersect with census blocks where the black population is above 50%?

In [47]:
%%bash
sudo -u postgres psql -d gis -P pager=off -c "

SELECT AVG(o.avg_u_kbps) AS avg_upload_speed
FROM ookla_nyc o
JOIN nyc_subway_stations s
  ON ST_DWithin(o.geom, s.geom, 200)
JOIN nyc_census_blocks b
  ON ST_Intersects(o.geom, b.geom)
WHERE s.routes LIKE '%L%'
  AND b.popn_total > 0
  AND b.popn_black::float / b.popn_total > 0.5;

"

 avg_upload_speed  
-------------------
 140603.2014051522
(1 row)

